<a href="https://colab.research.google.com/github/borbysh/quantum-barren-plateaus/blob/main/Tutorials/Barren_pennylane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
%matplotlib inline
!pip install pennylane

In [32]:
import sys
sys.path.append('..')
import pennylane.numpy as np
import matplotlib.pyplot as plt
import pennylane as qml

In [33]:
def global2local( Hamiltonian ):

    num_wires = len(Hamiltonian.wires)

    coeff = []
    ops   = []

    coeff_local = Hamiltonian.coeffs

    for i, hamiltonian in enumerate(Hamiltonian.ops):

        if int(hamiltonian.num_wires)== -1 or int(hamiltonian.num_wires)==1:
            ops.append( qml.Identity(range(num_wires)) )
            coeff.append( coeff_local[i] )

        else:
            pauli_local = hamiltonian.obs

            for pauli in pauli_local:
                ops.append( pauli )
                coeff.append( coeff_local[i]/num_wires  )

            ops.append( qml.Identity(range(num_wires)) )
            coeff.append( coeff_local[i]*(num_wires - hamiltonian.num_wires)/num_wires )

    return qml.Hamiltonian( np.array(coeff), ops).simplify()

def test_hamiltonian( num_wires ):

    Paulis = qml.Hamiltonian( [0.5, 0.5], [qml.Identity(0), qml.PauliZ(0) ] )
    for i in range(1,num_wires):
        zero = qml.Hamiltonian( [0.5, 0.5], [qml.Identity(i), qml.PauliZ(i) ] )
        Paulis = Paulis@zero

    Hamiltonian = qml.Hamiltonian( list(-Paulis.coeffs)+[1], Paulis.ops+[ qml.Identity(0) ] )

    return Hamiltonian.simplify()


def test_hamiltonian_2( num_wires, coeff ):

    obs = [ qml.pauli.string_to_pauli_word(num_wires*'X'),
            qml.pauli.string_to_pauli_word(num_wires*'Y') ,
            qml.pauli.string_to_pauli_word(num_wires*'Z') ]

    return qml.Hamiltonian(coeff, obs)

In [34]:
def VQE( Hamiltonian,
            ansatz,
            params,
            optimizer,
            max_iterations = 100,
            conv_tol       = None,
            device         = "default.qubit",
            shots          = None,
            callback       = None
             ):

    num_wires = len(Hamiltonian.wires)
    dev = qml.device(device, wires=num_wires, shots=shots)
    @qml.qnode(dev)
    def cost_func(params):
        ansatz(params)
        return qml.expval(Hamiltonian)

    # store the values of the cost function
    energy_k = [cost_func(params)]

    # store the values of the circuit parameter
    params_k = [params]

    for n in range(max_iterations):
        params, prev_energy = optimizer.step_and_cost(cost_func, params)

        energy_k.append(cost_func(params))
        params_k.append(params)

        if callback is not None:
            callback(params)

        # if n % 2 == 0:
            # print(f"Step = {n},  Energy = {energy_k[-1]:.8f} Ha")

        if conv_tol is not None:
            conv = np.abs(energy_k[-1] - prev_energy)
            if conv <= conv_tol:
                break

    return params_k, energy_k

In [35]:
def cnot_layer( n_qbits, n_cnot='Full_connect' ):
    """
    Create an entangling layer circuit.

    Input:
    n_qbits (int): Number of qubits of the circuit
    n_cnot (int, list, str):  Specify the type of entangling layer:
        str = 'Full_connect' ; that means the circuit is full fill with CNOT's gates.
        int = Number of CNOT's to implement from the first qubit to the last of the first half of qubits in descendet order.
        list= List of pair values e.g. [[1,2], [7,6]], first element of each list is control qubit e.g 1, 7 and the second the target qubit e.g 2, 6.

    Output:
    (QuantumCircuit): Entangling layer
    """
    sysA = int( len(n_qbits)/2 )

    if isinstance( n_cnot, int ):
        for i in range( n_cnot ):
            qml.CNOT( wires=[ n_qbits[i] , n_qbits[i + sysA] ] )

    elif n_cnot == 'Full_connect':
        for i in range( sysA ):
            qml.CNOT( wires=[ n_qbits[i] , n_qbits[i + sysA] ] )

    else:
        n_cnot = np.shape( n_cnot )[0]
        for i in range( n_cnot ):
            qml.CNOT( wires=[ n_cnot[i][0] , n_cnot[i][1] ] )


def SCL( params, n_qbits, n_qbits_crz=2, deep=1 ):
    """
    Schmidt Coefficient Layer (SCL) that performs a Schmidth decomposition or a basis change.

    Input:
    params (list):      Parameters to initialize the circuit.
    n_qbits (iterable): Qubits of the layer.
    n_qbits_crz (list): Qubits in the control Z gate. Default (2) is for 1 control qubit and 1 target qubit.
    deep (int):         Times that the circuit is repeated.
    name (str):         Name of the circuit.

    Output:
    (QuantumCircuit): A Parametric Quantum Circuit.
    """

    qubits= len( n_qbits )
    par = -(qubits % 2) # parity test: -1 for odd, 0 for even

    for i in range( qubits ):
        qml.RY( params[i], wires=[ n_qbits[i] ] )

    cc = -1
    for _ in range( 1, deep+1 ):

        for i in range( 0 , qubits + par, n_qbits_crz ):
            for l in range( 1, n_qbits_crz ):
                cc += 1
                qml.CZ( wires=[ n_qbits[i], n_qbits[i+l] ] )

        for i in range( qubits ):
            cc += 1
            qml.RY( params[ qubits + cc ], wires=[ n_qbits[i] ] )

        for i in range( 1, qubits-1, n_qbits_crz ):
            for l in range( 1, n_qbits_crz ):
                cc += 1
                qml.CZ( wires=[ n_qbits[i], n_qbits[i+l] ] )
                qml.RY( params[ qubits + cc ], wires=[ n_qbits[i] ] )
                qml.RY( params[ qubits + cc ], wires=[ n_qbits[i + l] ] )


def ansatz_constructor( n_qbits, unitaries=[SCL, SCL, SCL], n_qb_crz=[2,2,2], deep= [1,1,1], n_cnot='Full_connect' ):
    """
    The State Efficient Ansatz parametric quantum circuit (PQC) that perform a schmith decomposition, entanglement and a basis change.
    The ansatz circuit is made of three PQC's and one entangling layer.
    The input parameters will define this four gates of the ansatz circuit U_1, V, U_2 and U_3.

    Input:
    n_qbits (int):            Number of qubits.
    unitaries (list):         Containing the three PQC's that conform the SEA.
    n_qbits_crz (list):       Qubits in the control Z gate for each PQC. Default ([2,2,2]) is for 1 control qubit and 1 target qubit for each PQC.
    deep (list):              Times that each circuit is repeated.
    n_cnot (int, list, str):  Specify the type of entangling layer:
        str = 'Full_connect' ; that means the circuit is full fill with CNOT's gates.
        int = Number of CNOT's to implement from the first qubit to the last of the first half of qubits in descendet order.
        list= List of pair values e.g. [[1,2], [7,6]], first element of each list is control qubit e.g 1, 7 and the second the target qubit e.g 2, 6.

    Output:
    (QuantumCircuit): The State Efficient Ansatz parametric quantum circuit (PQC).
    """

    def circuit(params):
        unitaries[0]( params[0], range( int(n_qbits/2) ), n_qbits_crz=int(n_qb_crz[0]), deep=deep[0] )
        cnot_layer( range(n_qbits), n_cnot=n_cnot )
        unitaries[1]( params[1], range( int(n_qbits/2) ), n_qbits_crz=int(n_qb_crz[1]), deep=deep[1] )
        unitaries[2]( params[2], range( int(n_qbits/2), n_qbits ), n_qbits_crz=int(n_qb_crz[2]), deep=deep[2] )

    return circuit

We provides an early implementation of the local-global VQE and the State Efficient Ansatz for __Penylane__. We are going to optimize the following hamiltonian.

In [36]:
num_wires = 6
hamiltonian = test_hamiltonian_2( num_wires, [0,0,1] )
print( hamiltonian )

0 * (X(0) @ X(1) @ X(2) @ X(3) @ X(4) @ X(5)) + 0 * (Y(0) @ Y(1) @ Y(2) @ Y(3) @ Y(4) @ Y(5)) + 1 * (Z(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5))


Calculating the local Hamiltonian.

In [37]:
hamiltonian_local = global2local( hamiltonian )
print(hamiltonian_local)

AttributeError: 'Prod' object has no attribute 'obs'

Creating the variational circuit.

In [38]:
def ansatz(params, num_layers=0):

    num_wires = len(params)//(2*num_layers+2)

    K = 0
    for i in range(num_layers):
        qml.broadcast( qml.RY, wires=range(num_wires), pattern='single', parameters=params[K:K+num_wires] )
        qml.broadcast( qml.RZ, wires=range(num_wires), pattern='single', parameters=params[K+num_wires:K+2*num_wires] )
        qml.broadcast( qml.CNOT, wires=range(num_wires), pattern='ring')
        K = K+2*num_wires

    qml.broadcast( qml.RY, wires=range(num_wires), pattern='single', parameters=params[K:K+num_wires] )
    qml.broadcast( qml.RZ, wires=range(num_wires), pattern='single', parameters=params[K+num_wires:K+2*num_wires] )

Choosing SPSA as optimizer.

In [39]:
optimizer = qml.SPSAOptimizer(maxiter=1000)
maxiter   = 300

Defining a device.

In [29]:
dev = qml.device("default.qubit", wires=num_wires)
@qml.qnode(dev)
def cost_func(params):
    ansatz(params)
    return qml.expval(hamiltonian)

Runing the VQEs.

In [40]:
np.random.seed(100)
params_in = np.random.randn( 2*num_wires )*np.pi
params_out, energies_global = VQE( hamiltonian, ansatz, params_in, optimizer, shots=2**6, max_iterations=maxiter )

AttributeError: module 'pennylane' has no attribute 'broadcast'

In [41]:
np.random.seed(100)
params_in = np.random.randn( 2*num_wires )*np.pi
params_out, energies_local  = VQE( hamiltonian_local, ansatz, params_in, optimizer, shots=2**6, max_iterations=maxiter//2)
energies_global_from_local = [ cost_func(x) for x in params_out ]

NameError: name 'hamiltonian_local' is not defined

In [42]:
np.random.seed(100)
params_in = np.random.randn( 6*num_wires )*np.pi
params_out, energies_global_after_local = VQE( hamiltonian, ansatz, params_out[-1], optimizer, shots=2**6, max_iterations=maxiter//2 )

NameError: name 'params_out' is not defined

In [43]:
plt.plot( energies_global, 'o' )
plt.plot( energies_global_from_local, 'o' )
plt.plot( range(maxiter//2,maxiter+1), energies_global_after_local, 'o' )
plt.legend([ 'Standard', 'local', 'global'])


NameError: name 'energies_global' is not defined

We also implement the Efficient State Ansatz.

In [44]:
n_qbits = num_wires
reps    = 1
SEA = ansatz_constructor(n_qbits , deep=[reps,reps,reps])

num_params_SEA = ( reps +1 )*n_qbits + reps*(n_qbits - 2)

params_SEA = [ np.random.randn( num_params_SEA )*np.pi for _ in range(3) ]

In [45]:
par, results_SEA = VQE(hamiltonian, SEA, params_SEA, optimizer, shots=2**6, max_iterations=maxiter)

In [46]:
plt.plot( energies_global, label='SU' )
plt.plot( results_SEA, label='SEA' )
plt.legend(  )

NameError: name 'energies_global' is not defined

Our current implementation is not able to outperforme barren plateaus. We think that improve our VQE and the classical optimizer is mandatory to obtain better results. This is a first step to have this routines in Pennylane.